In [11]:
import torch
import tensorflow as tf
from langchain_community.document_loaders import PyPDFLoader, TextLoader

2024-04-12 10:02:24.517893: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-12 10:02:24.546960: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
DEBUG:tensorflow:Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
2024-04-12 10:02:24.940374: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Crea

In [12]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("using CPU")

using GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [3]:
from ragatouille import RAGPretrainedModel
RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

In [26]:
loader = PyPDFLoader("hallucination_paper.pdf")
pages = loader.load_and_split()

In [8]:
len(data_pages)

4602

In [12]:
len(pages)

89

In [27]:
full_document = ""

for page in pages:
  full_document += page.page_content

In [28]:
print(full_document)

A Survey on Hallucination in Large Language Models:
Principles, Taxonomy, Challenges, and Open Questions
Lei Huang1∗, Weijiang Yu2∗, Weitao Ma1, Weihong Zhong1
Zhangyin Feng1, Haotian Wang1, Qianglong Chen2, Weihua Peng2
Xiaocheng Feng1†, Bing Qin1, Ting Liu1
1Harbin Institute of Technology, Harbin, China
2Huawei Inc., Shenzhen, China
{lhuang,wtma,whzhong,zyfeng,xcfeng†,qinb,tliu}@ir.hit.edu.cn
{weijiangyu8,wanght1998,chenqianglong.ai,pengwh.hit}@gmail.com
Abstract
The emergence of large language models
(LLMs) has marked a significant breakthrough
in natural language processing (NLP), leading
to remarkable advancements in text understand-
ing and generation. Nevertheless, alongside
these strides, LLMs exhibit a critical tendency
to produce hallucinations, resulting in content
that is inconsistent with real-world facts or user
inputs. This phenomenon poses substantial
challenges to their practical deployment and
raises concerns over the reliability of LLMs in
real-world scenarios, which

In [29]:
type(full_document)

str

In [30]:
with torch.device(device):
    RAG.index(collection=[full_document], 
              index_name="hallucination_paper", 
              max_document_length=512, 
              split_documents=True)

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: A Survey on Hallucination in Large Language Mod...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Subsequently, we present a
comprehensive overvi...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: These
hallucinations are further categorized in...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Correct Answer: Neil Armstrong was the ﬁrst
per...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Our in-depth
analysis specifically targets the ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: While
these works have probed into LLM hallucin...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Concluding, we delve
into the challenges and op...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Analyzing
these stages provides insight into ha...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Furthermore, recent studies (Chung

New index_name received! Updating current index_name (hull_data1) to hallucination_paper
---- WARNING! You are using PLAID with an experimental replacement for FAISS for greater compatibility ----
This is a behaviour change from RAGatouille 0.8.0 onwards.
This works fine for most users and smallish datasets, but can be considerably slower than FAISS and could cause worse results in some situations.
If you're confident with FAISS working on your machine, pass use_faiss=True to revert to the FAISS-using behaviour.
--------------------


[Apr 12, 11:10:29] #> Note: Output directory .ragatouille/colbert/indexes/hallucination_paper already exists


[Apr 12, 11:10:29] #> Will delete 10 files already at .ragatouille/colbert/indexes/hallucination_paper in 20 seconds...


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/artifact.metadata HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0

[Apr 12, 11:10:51] [0] 		 #> Encoding 139 passages..
[Apr 12, 11:10:53] [0] 		 avg_doclen_est = 366.3165588378906 	 len(local_sample) = 139
[Apr 12, 11:10:53] [0] 		 Creating 2,048 partitions.
[Apr 12, 11:10:53] [0] 		 *Estimated* 50,918 embeddings.
[Apr 12, 11:10:53] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/hallucination_paper/plan.json ..
used 20 iterations (0.641s) to cluster 48373 items into 2048 clusters
[0.032, 0.033, 0.031, 0.032, 0.031, 0.034, 0.031, 0.03, 0.032, 0.032, 0.029, 0.029, 0.032, 0.032, 0.031, 0.031, 0.03, 0.033, 0.031, 0.032, 0.032, 0.032, 0.032, 0.032, 0.031, 0.032, 0.032, 0.033, 0.033, 0.033, 0.031, 0.035, 0.032, 0.031, 0.031, 0.029, 0.032, 0.031, 0.032, 0.036, 0.034, 0.034, 0.031, 0.033, 0.03, 0.031, 0.03, 0.035, 0.032, 0.032, 0.032, 0.033, 0.035, 0.031, 0.031, 0.033, 0.038, 0.032, 0.037, 0.031, 0.032, 0.033, 0.035, 0.033, 0.032, 0.033, 0.033, 0.031, 0.029, 0.031, 0.033, 0.03, 0.033, 0.032, 0.034, 0.031, 0.031, 0.033, 0.032, 0.035, 0.035

0it [00:00, ?it/s]

...

1it [00:01,  1.51s/it]
100%|████████████████████████████| 1/1 [00:00<00:00, 122.13it/s]

[Apr 12, 11:10:55] #> Optimizing IVF to store map from centroids to list of pids..
[Apr 12, 11:10:55] #> Building the emb2pid mapping..
[Apr 12, 11:10:55] len(emb2pid) = 50918



100%|████████████████████| 2048/2048 [00:00<00:00, 10913.05it/s]

[Apr 12, 11:10:55] #> Saved optimized IVF to .ragatouille/colbert/indexes/hallucination_paper/ivf.pid.pt


Done indexing!


In [31]:
with torch.device(device):
    search = RAG.search(query="What is Hallucination?", k=3)

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/artifact.metadata HTTP/1.1" 200 0


Loading searcher for index hallucination_paper for the first time... This may take a few seconds


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/artifact.metadata HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0

[Apr 12, 11:11:00] #> Loading codec...
[Apr 12, 11:11:00] #> Loading IVF...
[Apr 12, 11:11:00] #> Loading doclens...


100%|████████████████████████████| 1/1 [00:00<00:00, 205.93it/s]

[Apr 12, 11:11:00] #> Loading codes and residuals...



100%|█████████████████████████████| 1/1 [00:00<00:00, 19.39it/s]


Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . What is Hallucination?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2054,  2003,  2534, 14194, 12758,  1029,   102,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')



In [32]:
search

[{'content': 'Such\nintegration of human feedback into the training\nloop has proven effective in enhancing the align-\nment of LLMs, guiding them toward producing\nhigh-quality and harmless responses.\n2.3 Hallucinations in Large Language Models\nThe concept of hallucination traces its roots to the\nfields of pathology and psychology and is defined\nasthe perception of an entity or event that is ab-\nsent in reality (Macpherson and Platchias, 2013).\nWithin the realm of NLP, hallucination is typically\nreferred to as a phenomenon in which the gen-\nerated content appears nonsensical or unfaithful\nto the provided source content (Filippova, 2020;\nMaynez et al., 2020). This concept bears a loose\nresemblance to the phenomenon of hallucination\nobserved in human psychology. Generally, halluci-\nnations in natural language generation tasks can be\ncategorized into two primary types: intrinsic hallu-\ncination andextrinsic hallucination (Huang et al.,\n2021; Li et al., 2022b; Ji et al., 2

## Using Langchain Retrieval

In [33]:
retriever = RAG.as_langchain_retriever(k=3)

In [34]:
retriever.invoke("What is hallucination?")

[Document(page_content='Such\nintegration of human feedback into the training\nloop has proven effective in enhancing the align-\nment of LLMs, guiding them toward producing\nhigh-quality and harmless responses.\n2.3 Hallucinations in Large Language Models\nThe concept of hallucination traces its roots to the\nfields of pathology and psychology and is defined\nasthe perception of an entity or event that is ab-\nsent in reality (Macpherson and Platchias, 2013).\nWithin the realm of NLP, hallucination is typically\nreferred to as a phenomenon in which the gen-\nerated content appears nonsensical or unfaithful\nto the provided source content (Filippova, 2020;\nMaynez et al., 2020). This concept bears a loose\nresemblance to the phenomenon of hallucination\nobserved in human psychology. Generally, halluci-\nnations in natural language generation tasks can be\ncategorized into two primary types: intrinsic hallu-\ncination andextrinsic hallucination (Huang et al.,\n2021; Li et al., 2022b; Ji

## Using OpenAI

In [5]:
import os
openai_api_key = os.environ["OPENAI_API_KEY"]

In [35]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}"""
)

llm = ChatOpenAI()
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/home/jerryola1/.local/lib/python3.10/site-packages/certifi/cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/home/jerryola1/.local/lib/python3.10/site-packages/certifi/cacert.pem'


In [36]:
response = retrieval_chain.invoke({"input": "What is hallucination?"})

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': 'Answer the following question based only on the provided context:\n\n<context>\nSuch\nintegration of human feedback into the training\nloop has proven effective in enhancing the align-\nment of LLMs, guiding them toward producing\nhigh-quality and harmless responses.\n2.3 Hallucinations in Large Language Models\nThe concept of hallucination traces its roots to the\nfields of pathology and psychology and is defined\nasthe perception of an entity or event that is ab-\nsent in reality (Macpherson and Platchias, 2013).\nWithin the realm of NLP, hallucination is typically\nreferred to as a phenomenon in which the gen-\nerated content appears nonsensical or unfaithful\nto the provided source content (Filippova, 2020;\nMaynez et al., 2020). This concept bears a loose\nresemblance to the phenomenon of hallucination\nobserved in human psychology. Gener

In [37]:
response["answer"]

'Hallucination, in the context of large language models (LLMs), refers to a phenomenon where the generated content appears nonsensical or unfaithful to the provided source content. It can be categorized into intrinsic hallucination, which pertains to outputs conflicting with the source content, and extrinsic hallucination, which refers to LLM generations that cannot be verified from the source content.'

# Hull university 

In [38]:
load_hull_data = TextLoader("data.txt")
hull_data_pages = load_hull_data.load_and_split()

In [39]:
hull_document = ""
for hull_page in hull_data_pages:
    hull_document += hull_page.page_content

print(hull_document)
type(hull_document)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



str

### Create Index

In [41]:
with torch.device(device):
    RAG.index(collection=[hull_document], 
              index_name="hull_data2", 
              max_document_length=512, 
              split_documents=True, 
              use_faiss=False)

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: It's not too late to apply

Study at Hull in 20...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Find out more

Life-changing research

Our rese...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Posted on 22 January 2024

News
               ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Postgraduate





Other





Sorry, we couldn't...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Join us in tackling the big issues – climate ch...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: support and outcomes for cancer patients

Reduc...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 2. All new students are guaranteed a single-per...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Book your place on to one of our Open Days now....
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: With so many people with different

[Apr 12, 11:19:09] [0] 		 avg_doclen_est = 386.1312255859375 	 len(local_sample) = 8,375
[Apr 12, 11:19:14] [0] 		 Creating 16,384 partitions.
[Apr 12, 11:19:14] [0] 		 *Estimated* 3,233,849 embeddings.
[Apr 12, 11:19:14] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/hull_data2/plan.json ..
used 20 iterations (14.6564s) to cluster 3183849 items into 16384 clusters
[0.037, 0.038, 0.038, 0.034, 0.036, 0.039, 0.035, 0.034, 0.033, 0.036, 0.034, 0.037, 0.036, 0.036, 0.036, 0.036, 0.032, 0.034, 0.034, 0.035, 0.036, 0.037, 0.035, 0.036, 0.034, 0.036, 0.039, 0.038, 0.036, 0.038, 0.038, 0.041, 0.041, 0.035, 0.035, 0.033, 0.036, 0.037, 0.036, 0.04, 0.037, 0.035, 0.037, 0.035, 0.036, 0.036, 0.036, 0.041, 0.038, 0.035, 0.033, 0.037, 0.038, 0.036, 0.035, 0.037, 0.041, 0.037, 0.044, 0.035, 0.036, 0.038, 0.038, 0.036, 0.038, 0.036, 0.036, 0.037, 0.035, 0.035, 0.036, 0.034, 0.035, 0.037, 0.036, 0.036, 0.036, 0.039, 0.038, 0.038, 0.041, 0.038, 0.036, 0.037, 0.039, 0.036, 0.036, 0.0

0it [00:00, ?it/s]

[Apr 12, 11:19:34] [0] 		 #> Encoding 8375 passages..


1it [01:56, 116.12s/it]
100%|█████████████████████████████| 1/1 [00:00<00:00, 12.42it/s]

[Apr 12, 11:21:30] #> Optimizing IVF to store map from centroids to list of pids..
[Apr 12, 11:21:30] #> Building the emb2pid mapping..
[Apr 12, 11:21:30] len(emb2pid) = 3233849



100%|██████████████████| 16384/16384 [00:01<00:00, 16226.17it/s]


[Apr 12, 11:21:31] #> Saved optimized IVF to .ragatouille/colbert/indexes/hull_data2/ivf.pid.pt
Done indexing!


### Load index

In [42]:
index_name = "hull_data2"
results = RAG.search(query="How much is AI and DS school feels?", k=1, index_name=index_name)
print(results)

DEBUG:urllib3.connectionpool:Resetting dropped connection: huggingface.co
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/artifact.metadata HTTP/1.1" 200 0


Loading searcher for index hull_data2 for the first time... This may take a few seconds


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/artifact.metadata HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0

[Apr 12, 11:21:49] #> Loading codec...
[Apr 12, 11:21:49] #> Loading IVF...
[Apr 12, 11:21:49] #> Loading doclens...


100%|████████████████████████████| 1/1 [00:00<00:00, 160.78it/s]

[Apr 12, 11:21:49] #> Loading codes and residuals...



100%|█████████████████████████████| 1/1 [00:01<00:00,  1.16s/it]

Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . How much is AI and DS school feels?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2129,  2172,  2003,  9932,  1998, 16233,  2082,  5683,
         1029,   102,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

[{'content': 'The design of the new building offers an immersive, contemporary, and high-quality digital setting whilst keeping its environmental impact to a minimum.Dr Kevin Pimbblet, Director of the new Centre, said: “The past 12 months has shown how quickly the field of Artificial Intelligence can develop. With some jobs and careers now in danger of being replaced 

In [43]:
retriever = RAG.as_langchain_retriever(k=3)

In [44]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import gradio as gr

In [10]:
# prompt = ChatPromptTemplate.from_template(
#     """Answer the following question based only on the provided context:

# <context>
# {context}
# </context>

# Question: {input}"""
# )

# llm = ChatOpenAI()
# document_chain = create_stuff_documents_chain(llm, prompt)
# retrieval_chain = create_retrieval_chain(retriever, document_chain)

# # response = retrieval_chain.invoke({"input": "Who Is Dr Temitayo Fagbola?"})
# response = retrieval_chain.invoke({"input": "In one sentence, who is dr temitayo?"})

In [46]:
response["answer"]

'Dr. Brian May is a founding member of Queen, guitarist, renowned songwriter, skilled producer, beloved performer, Doctor of Astrophysics, and Lecturer in Mechanical Engineering at the University of Hull.'

# Scrap Data from URL

In [ ]:
# def get_sitemap_urls(sitemap_url):
#     response = requests.get(sitemap_url)
#     sitemap = response.content
#     root = ET.fromstring(sitemap)
#     urls = [url.text for url in root.findall('.//{http://www.sitemaps.org/schemas/sitemap/0.9}loc')]
#     return urls

# # Get URLs from sitemap
# sitemap_url = 'https://www.hull.ac.uk/sitemap.xml'
# urls = get_sitemap_urls(sitemap_url)

# from langchain.document_loaders import UnstructuredURLLoader
# while tf.device(device):
#     loaders = UnstructuredURLLoader(urls=urls)
#     data = loaders.load()

# Save & Load Data

In [ ]:
# import json
# # Prepare data for saving
# data_to_save = [{'page_content': doc.page_content, 'metadata': doc.metadata} for doc in data]

# # Save data to a JSON file
# with open('data.json', 'w') as file:
#     json.dump(data_to_save, file)

# data_str = '\n'.join(doc.page_content for doc in data)

# # Save the string to a text file
# with open('data.txt', 'w') as file:
#     file.write(data_str)

# Gradio

In [15]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("using CPU")


import gradio as gr
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader, PyPDFLoader
from ragatouille import RAGPretrainedModel
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from ragatouille import RAGPretrainedModel
RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)


#Load and split the data
# load_hull_data = TextLoader("data.txt")
# hull_data_pages = load_hull_data.load_and_split()

# hull_document = ""
# for hull_page in hull_data_pages:
#     hull_document += hull_page.page_content

# with torch.device(device):
#     RAG.index(collection=[hull_document], 
#               index_name="hull_data", 
#               max_document_length=512, 
#               split_documents=True)

#Specify the index name
index_name = "hull_data"

# Create a retriever
try:
    retriever = RAG.as_langchain_retriever(index_name=index_name, k=3)
    logger.info(f"Retriever created with index_name: {index_name}")
except AssertionError as e:
    logger.warning(f"Index '{index_name}' already exists. Skipping index creation.")
    retriever = RAG.as_langchain_retriever(index_name=index_name, k=3)

# Create a prompt template
prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}"""
)

# Create a language model
llm = ChatOpenAI(streaming=True, max_tokens=150)
# llm = ChatOllama(model="llama2")

# Create a document chain and retrieval chain
document_chain = create_stuff_documents_chain(llm, prompt)
# chain = prompt | llm | StrOutputParser()
retrieval = create_retrieval_chain(retriever, document_chain)

# response = retrieval_chain.invoke({"input": "What is dr fagbola??"})
# print(response)

# results = RAG.search(query="What is dr fagbola?", k=3)

def ask_question(question):
    response = retrieval_chain.invoke({"input": question})
    return response["answer"]

iface = gr.Interface(
    fn=ask_question,
    inputs=gr.Textbox(lines=2, placeholder="Ask a question..."),
    outputs="text",
    title="Question Answering with Langchain and RAGatouille",
    description="Type your question and press Enter to get an answer based on the loaded data.",
)

iface.launch()

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/artifact.metadata HTTP/1.1" 200 0


using GPU: NVIDIA GeForce RTX 4060 Laptop GPU


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/artifact.metadata HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
INFO:__main__:Retriever created with index_name: hull_data
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/home/jerryola1/.local/lib/python3.10/site-packages/certifi/cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_ver

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False


DEBUG:httpx:load_verify_locations cafile='/home/jerryola1/.local/lib/python3.10/site-packages/certifi/cacert.pem'
DEBUG:httpcore.connection:connect_tcp.started host='api.gradio.app' port=443 local_address=None timeout=5 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7f3b1cecbdc0>
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7f3b1cecbc40>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x7f3b1ec12940> server_hostname='api.gradio.app' timeout=3
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x7f3b41eaab40> server_hostname='api.gradio.app' timeout=5
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7f3b1cec9480>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x7f3b410451c0> server_hostnam

# StreamLit

In [1]:
# import streamlit as st

# from langchain.text_splitter import CharacterTextSplitter
# from langchain.document_loaders import TextLoader, PyPDFLoader
# from ragatouille import RAGPretrainedModel
# from langchain.chains import create_retrieval_chain
# from langchain.chains.combine_documents import create_stuff_documents_chain
# from langchain.prompts import ChatPromptTemplate
# from langchain.llms import OpenAI
# from langchain_openai import ChatOpenAI
# from ragatouille import RAGPretrainedModel

# RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

# # Load and split the data
# load_hull_data = TextLoader("data.txt")
# hull_data_pages = load_hull_data.load_and_split()
# hull_document = ""
# for hull_page in hull_data_pages:
#     hull_document += hull_page.page_content

# # Create a RAG model
# # RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

# # Specify the index name
# index_name = "hull_data"

# # Create a retriever
# # retriever = RAG.as_langchain_retriever(k=3
# # Create a retriever
# try:
#     retriever = RAG.as_langchain_retriever(index_name=index_name, k=3)
# except Exception as e:
#     logger.error(f"Error creating retriever: {e}")
#     raise

# # Create a prompt template
# prompt = ChatPromptTemplate.from_template(
#     """Answer the following question based only on the provided context:

#     <context>
#     {context}
#     </context>

#     Question: {input}"""
# )

# # Create a language model
# llm = ChatOpenAI(streaming=True, max_tokens=150)

# # llm = Ollama

# # Create a document chain and retrieval chain
# document_chain = create_stuff_documents_chain(llm, prompt)
# retrieval_chain = create_retrieval_chain(retriever, document_chain)

# def ask_question(question):
#     response = retrieval_chain.invoke({"input": question})
#     return response["answer"]

# st.title("Question Answering with Langchain and RAGatouille")
# st.write("Type your question and press Enter to get an answer based on the loaded data.")

# question = st.text_input("Ask a question...")

# if question:
#     answer = ask_question(question)
#     st.write(answer)

In [ ]:
RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

In [20]:
import requests

def get_wikipedia_page(title: str):
    """
    Retrieve the full text content of a Wikipedia page.

    :param title: str - Title of the Wikipedia page.
    :return: str - Full text content of the page as raw string.
    """
    # Wikipedia API endpoint
    URL = "https://en.wikipedia.org/w/api.php"

    # Parameters for the API request
    params = {
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "extracts",
        "explaintext": True,
    }

    # Custom User-Agent header to comply with Wikipedia's best practices
    headers = {"User-Agent": "RAGatouille_tutorial/0.0.1 (ben@clavie.eu)"}

    response = requests.get(URL, params=params, headers=headers)
    data = response.json()

    # Extracting page content
    page = next(iter(data["query"]["pages"].values()))
    return page["extract"] if "extract" in page else None

In [21]:
full_document = get_wikipedia_page("Hayao_Miyazaki")

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): en.wikipedia.org:443
DEBUG:urllib3.connectionpool:https://en.wikipedia.org:443 "GET /w/api.php?action=query&format=json&titles=Hayao_Miyazaki&prop=extracts&explaintext=True HTTP/1.1" 200 None


In [26]:
RAG.index(
    collection=[full_document],
    index_name="Miyazaki-123",
    max_document_length=180,
    split_documents=True,
)
results = RAG.search(query="What animation studio did Miyazaki found?", k=3)

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Hayao Miyazaki (宮崎 駿 or 宮﨑 駿, Miyazaki Hayao, J...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: During his early years at Toei Animation he wor...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: He joined Tokyo Movie Shinsha in 1979 to direct...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: The films were met with critical and commercial...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Miyazaki's later films—Howl's Moving Castle (20...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: The protagonists of his films are often strong ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: == Early life ==
Hayao Miyazaki was born on Jan...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: According to Miyazaki, his father often told hi...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: In 1944, when he was three years o

New index_name received! Updating current index_name (Miyazaki-123) to Miyazaki-123
---- WARNING! You are using PLAID with an experimental replacement for FAISS for greater compatibility ----
This is a behaviour change from RAGatouille 0.8.0 onwards.
This works fine for most users and smallish datasets, but can be considerably slower than FAISS and could cause worse results in some situations.
If you're confident with FAISS working on your machine, pass use_faiss=True to revert to the FAISS-using behaviour.
--------------------


[Apr 14, 16:53:55] #> Note: Output directory .ragatouille/colbert/indexes/Miyazaki-123 already exists


[Apr 14, 16:53:55] #> Will delete 10 files already at .ragatouille/colbert/indexes/Miyazaki-123 in 20 seconds...


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/artifact.metadata HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /colbert-ir/colbertv2.0

[Apr 14, 16:54:17] [0] 		 #> Encoding 81 passages..
[Apr 14, 16:54:17] [0] 		 avg_doclen_est = 129.88888549804688 	 len(local_sample) = 81
[Apr 14, 16:54:17] [0] 		 Creating 1,024 partitions.
[Apr 14, 16:54:17] [0] 		 *Estimated* 10,520 embeddings.
[Apr 14, 16:54:17] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/Miyazaki-123/plan.json ..
used 18 iterations (0.107s) to cluster 9995 items into 1024 clusters
[0.037, 0.039, 0.039, 0.033, 0.032, 0.036, 0.033, 0.037, 0.034, 0.035, 0.035, 0.037, 0.035, 0.039, 0.037, 0.038, 0.033, 0.034, 0.034, 0.035, 0.036, 0.035, 0.036, 0.034, 0.038, 0.035, 0.036, 0.034, 0.035, 0.035, 0.034, 0.036, 0.037, 0.032, 0.033, 0.033, 0.034, 0.034, 0.035, 0.042, 0.036, 0.041, 0.035, 0.033, 0.037, 0.035, 0.036, 0.035, 0.038, 0.035, 0.034, 0.035, 0.035, 0.038, 0.036, 0.038, 0.037, 0.037, 0.04, 0.031, 0.035, 0.035, 0.034, 0.033, 0.037, 0.038, 0.036, 0.036, 0.031, 0.034, 0.035, 0.034, 0.032, 0.037, 0.035, 0.034, 0.034, 0.037, 0.036, 0.036, 0.036, 0.0

0it [00:00, ?it/s]

...

[Apr 14, 16:54:19] #> Loading codec...
[Apr 14, 16:54:19] #> Loading IVF...
[Apr 14, 16:54:19] #> Loading doclens...


100%|████████████████████████████| 1/1 [00:00<00:00, 218.16it/s]

[Apr 14, 16:54:19] #> Loading codes and residuals...



100%|█████████████████████████████| 1/1 [00:00<00:00, 46.59it/s]


Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . What animation studio did Miyazaki found?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2054,  7284,  2996,  2106,  2771,  3148, 18637,  2179,
         1029,   102,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')



In [28]:
print(results)
retriever = RAG.as_langchain_retriever(k=3)
retriever.invoke("What animation studio did Miyazaki found?")

[{'content': 'In April 1984, Miyazaki opened his own office in Suginami Ward, naming it Nibariki.\n\n\n=== Studio Ghibli ===\n\n\n==== Early films (1985–1996) ====\nIn June 1985, Miyazaki, Takahata, Tokuma and Suzuki founded the animation production company Studio Ghibli, with funding from Tokuma Shoten. Studio Ghibli\'s first film, Laputa: Castle in the Sky (1986), employed the same production crew of Nausicaä. Miyazaki\'s designs for the film\'s setting were inspired by Greek architecture and "European urbanistic templates".', 'score': 25.90625, 'rank': 1, 'document_id': 'b4e7ccce-0263-47a6-9849-04a3211e4815', 'passage_id': 28}, {'content': 'Hayao Miyazaki (宮崎 駿 or 宮﨑 駿, Miyazaki Hayao, Japanese: [mijaꜜzaki hajao]; born January 5, 1941) is a Japanese animator, filmmaker, and manga artist. A co-founder of Studio Ghibli, he has attained international acclaim as a masterful storyteller and creator of Japanese animated feature films, and is widely regarded as one of the most accomplished

[Document(page_content='In April 1984, Miyazaki opened his own office in Suginami Ward, naming it Nibariki.\n\n\n=== Studio Ghibli ===\n\n\n==== Early films (1985–1996) ====\nIn June 1985, Miyazaki, Takahata, Tokuma and Suzuki founded the animation production company Studio Ghibli, with funding from Tokuma Shoten. Studio Ghibli\'s first film, Laputa: Castle in the Sky (1986), employed the same production crew of Nausicaä. Miyazaki\'s designs for the film\'s setting were inspired by Greek architecture and "European urbanistic templates".'),
 Document(page_content='Hayao Miyazaki (宮崎 駿 or 宮﨑 駿, Miyazaki Hayao, Japanese: [mijaꜜzaki hajao]; born January 5, 1941) is a Japanese animator, filmmaker, and manga artist. A co-founder of Studio Ghibli, he has attained international acclaim as a masterful storyteller and creator of Japanese animated feature films, and is widely regarded as one of the most accomplished filmmakers in the history of animation.\nBorn in Tokyo City in the Empire of Japa

In [30]:
# Chain

In [31]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}"""
)

llm = ChatOpenAI()

document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)
retrieval_chain.invoke({"input": "What animation studio did Miyazaki found?"})
for s in retrieval_chain.stream({"input": "What animation studio did Miyazaki found?"}):
    print(s.get("answer", ""), end="")

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/home/jerryola1/.local/lib/python3.10/site-packages/certifi/cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/home/jerryola1/.local/lib/python3.10/site-packages/certifi/cacert.pem'
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': 'Answer the following question based only on the provided context:\n\n<context>\nIn April 1984, Miyazaki opened his own office in Suginami Ward, naming it Nibariki.\n\n\n=== Studio Ghibli ===\n\n\n==== Early films (1985–1996) ====\nIn June 1985, Miyazaki, Takahata, Tokuma and Suzuki founded the animation production company Studio Ghibli, with funding from Tokuma Shoten. Studio Ghibli\'s first film, Laputa: Castle in the Sky (1986), employed the same production crew of

Miyazaki founded Studio Ghibli.